In [122]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [123]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report


In [124]:
def golden_cross_strategy1(moving_avgs_file, full_dataset_file, session_length=7):
    moving_avgs = pd.read_csv(moving_avgs_file, parse_dates=['datetime'])
    full_data = pd.read_csv(full_dataset_file, parse_dates=['datetime'])
    merged_data = pd.merge(moving_avgs, full_data[['datetime', 'Cumulative_Return']], 
                          on='datetime', how='left')
    
    merged_data['daily_return'] = merged_data['Cumulative_Return']
    
    merged_data['signal'] = 0
    merged_data['position'] = 0
    merged_data['strategy_return'] = 0.0
    
    in_session = False
    session_counter = 0
    session_buys = 0
    
    for i in range(1, len(merged_data)):
        sma5 = merged_data.at[i, 'SMA_5']
        sma20 = merged_data.at[i, 'SMA_20']
        sma50 = merged_data.at[i, 'SMA_200']
        
        buy_condition = (
            (sma5 > sma20 > sma50) or 
            (sma20 > sma5 > sma50) or 
            (sma5 > sma50 > sma20) or
            (sma20 > sma50)
            
        )
        
        if in_session:
            session_counter += 1
            if buy_condition:
                session_buys += 1
                merged_data.at[i, 'signal'] = 1
                merged_data.at[i, 'position'] = 1
            else:
                merged_data.at[i, 'signal'] = 0
                merged_data.at[i, 'position'] = 1
            
            if session_counter >= session_length and session_buys < (session_length/2):
                in_session = False
                merged_data.at[i, 'position'] = 0
        else:
            if buy_condition:
                in_session = True
                session_counter = 1
                session_buys = 1
                merged_data.at[i, 'signal'] = 1
                merged_data.at[i, 'position'] = 1
    
    merged_data['correct_prediction'] = (
        ((merged_data['daily_return'] < 0) & (merged_data['position'] == 0)) |  # True Negative (correctly avoided loss)
        ((merged_data['daily_return'] > 0) & (merged_data['position'] == 1))   # True Positive (correctly gained)
    )

    accuracy = merged_data['correct_prediction'].mean()

    y_true = (merged_data['daily_return'] > 0).astype(int) 
    y_pred = merged_data['position'] 

    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()

    report = classification_report(y_true, y_pred, target_names=['Avoid Loss (0)', 'Capture Gain (1)'])

    print("\nSTRATEGY PERFORMANCE:")
    print(f"Accuracy: {accuracy:.2%}")
    print("\nConfusion Matrix:")
    print(cm)
    print(f"\nTrue Negatives (Avoid Loss Correctly): {tn}")
    print(f"False Positives (Invested but Lost): {fp}")
    print(f"False Negatives (Missed Gains): {fn}")
    print(f"True Positives (Captured Gains): {tp}")
    print("\nClassification Report:")
    print(report)
    
    merged_data['strategy_return'] = merged_data['position'].shift(1) * merged_data['daily_return']
    merged_data['cumulative_strategy'] = merged_data['strategy_return'].cumsum()
    
    merged_data['perfect_return'] = merged_data['daily_return'].where(merged_data['daily_return'] > 0, 0)
    merged_data['cumulative_perfect'] = merged_data['perfect_return'].cumsum()
    
    total_strategy = merged_data['cumulative_strategy'].iloc[-1]
    total_perfect = merged_data['cumulative_perfect'].iloc[-1]
    efficiency = total_strategy / total_perfect if total_perfect > 0 else 0
    
    print("\nSTRATEGY PERFORMANCE:")
    print(f"Strategy Total Return: {total_strategy:.2f}")
    print(f"Perfect Strategy Total Return: {total_perfect:.2f}")
    print(f"Efficiency (Strategy/Perfect): {efficiency:.2%}")
    print(f"Days in Market: {merged_data['position'].sum()}/{len(merged_data)}")
    
    return merged_data


In [125]:
def golden_cross_strategy2(moving_avgs_file, full_dataset_file, session_length=7):

    moving_avgs = pd.read_csv(moving_avgs_file, parse_dates=['datetime'])
    full_data = pd.read_csv(full_dataset_file, parse_dates=['datetime'])
    merged_data = pd.merge(moving_avgs, full_data[['datetime', 'Cumulative_Return']], 
                          on='datetime', how='left')
    

    merged_data['daily_return'] = merged_data['Cumulative_Return']
    

    merged_data['signal'] = 0  # 0=sell, 1=buy
    merged_data['position'] = 0  # 0=out, 1=in
    merged_data['consecutive_sells'] = 0
    

    in_session = False
    days_in_session = 0
    
    for i in range(1, len(merged_data)):

        sma5 = merged_data.at[i, 'SMA_5']
        sma20 = merged_data.at[i, 'SMA_20']
        sma200 = merged_data.at[i, 'SMA_200']
        

        buy_condition = (
            (sma5 > sma20 > sma200) or 
            (sma20 > sma5 > sma200) or 
            (sma5 > sma200 > sma20) or
            (sma20 > sma200)
        )
        
        if buy_condition:
            merged_data.at[i, 'signal'] = 1
            merged_data.at[i, 'consecutive_sells'] = 0
        else:
            merged_data.at[i, 'signal'] = 0
            merged_data.at[i, 'consecutive_sells'] = merged_data.at[i-1, 'consecutive_sells'] + 1
        

        if not in_session:
            if buy_condition:
                in_session = True
                days_in_session = 1
                merged_data.at[i, 'position'] = 1
        else:
            days_in_session += 1
            
            if (merged_data.at[i, 'consecutive_sells'] >= 3) or (days_in_session >= session_length):
                in_session = False
                merged_data.at[i, 'position'] = 0
            else:
                merged_data.at[i, 'position'] = 1
    

    merged_data['correct_prediction'] = (
            ((merged_data['daily_return'] < 0) & (merged_data['position'] == 0)) |  # True Negative (correctly avoided loss)
            ((merged_data['daily_return'] > 0) & (merged_data['position'] == 1))   # True Positive (correctly gained)
        )

    accuracy = merged_data['correct_prediction'].mean()

    y_true = (merged_data['daily_return'] > 0).astype(int) 
    y_pred = merged_data['position'] 

    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()

    report = classification_report(y_true, y_pred, target_names=['Avoid Loss (0)', 'Capture Gain (1)'])

    print("\nSTRATEGY PERFORMANCE:")
    print(f"Accuracy: {accuracy:.2%}")
    print("\nConfusion Matrix:")
    print(cm)
    print(f"\nTrue Negatives (Avoid Loss Correctly): {tn}")
    print(f"False Positives (Invested but Lost): {fp}")
    print(f"False Negatives (Missed Gains): {fn}")
    print(f"True Positives (Captured Gains): {tp}")
    print("\nClassification Report:")
    print(report)

    merged_data['strategy_return'] = merged_data['position'].shift(1) * merged_data['daily_return']
    merged_data['cumulative_strategy'] = merged_data['strategy_return'].cumsum()
    

    merged_data['perfect_return'] = merged_data['daily_return'].where(merged_data['daily_return'] > 0, 0)
    merged_data['cumulative_perfect'] = merged_data['perfect_return'].cumsum()
    

    total_strategy = merged_data['cumulative_strategy'].iloc[-1]
    total_perfect = merged_data['cumulative_perfect'].iloc[-1]
    efficiency = total_strategy / total_perfect if total_perfect > 0 else 0
    
    print("\nREVISED STRATEGY PERFORMANCE:")
    print(f"Total Trading Days: {len(merged_data)}")
    print(f"Days in Position: {merged_data['position'].sum()} ({merged_data['position'].mean():.1%})")
    print(f"Strategy Return: {total_strategy:.2f}")
    print(f"Perfect Strategy Return: {total_perfect:.2f}")
    print(f"Efficiency: {efficiency:.2%}")
    print(f"Max Consecutive Days in Position: {(merged_data['position'].diff() == 1).cumsum().value_counts().max()}")
    
    return merged_data

In [126]:
def enhanced_sma_strategy(moving_avgs_file, full_dataset_file, session_length=7, threshold_pct=0.5, min_slope=0.1):

    data = pd.merge(
        pd.read_csv(moving_avgs_file, parse_dates=['datetime']),
        pd.read_csv(full_dataset_file, parse_dates=['datetime'])[['datetime', 'Cumulative_Return']],
        on='datetime'
    )
    
    data['daily_return'] = data['Cumulative_Return']
    data['sma5_slope'] = data['SMA_5'].pct_change() * 100 
    data['sma20_slope'] = data['SMA_20'].pct_change() * 100
    
    data['signal'] = 0
    data['position'] = 0
    data['consecutive_sells'] = 0
    
    for i in range(2, len(data)):
        sma5 = data.at[i, 'SMA_5']
        sma20 = data.at[i, 'SMA_20']
        sma50 = data.at[i, 'SMA_200']
        slope5 = data.at[i, 'sma5_slope']
        slope20 = data.at[i, 'sma20_slope']
        prev_sma5 = data.at[i-1, 'SMA_5']
        prev_sma20 = data.at[i-1, 'SMA_20']
        
        sma5_above_20 = (sma5 > sma20 * (1 + threshold_pct/100))
        sma20_above_50 = (sma20 > sma50 * (1 + threshold_pct/100))
        strong_slope = (slope5 > min_slope) and (slope20 > min_slope/2)
        crossing_up = (sma5 > sma20) and (prev_sma5 <= prev_sma20)
        
        buy_signal = sum([
            sma5_above_20,
            sma20_above_50, 
            strong_slope,
            crossing_up
        ]) >= 2
        
        sma5_below_20 = (sma5 < sma20 * (1 - threshold_pct/100))
        negative_slope = (slope5 < -min_slope)
        
        if buy_signal and not (sma5_below_20 or negative_slope):
            data.at[i, 'signal'] = 1
            data.at[i, 'consecutive_sells'] = 0
        else:
            data.at[i, 'signal'] = 0
            data.at[i, 'consecutive_sells'] = data.at[i-1, 'consecutive_sells'] + 1
    
    in_session = False
    for i in range(1, len(data)):
        if not in_session:
            if data.at[i, 'signal'] == 1:
                in_session = True
                data.at[i, 'position'] = 1
        else:
            if (data.at[i, 'consecutive_sells'] >= 3) or \
               ((i - data[data['position'].diff() == 1].index[-1]) >= session_length):
                in_session = False
                data.at[i, 'position'] = 0
            else:
                data.at[i, 'position'] = 1
    
    data['correct_prediction'] = (
        ((data['daily_return'] < 0) & (data['position'] == 0)) |  # True Negative (correctly avoided loss)
        ((data['daily_return'] > 0) & (data['position'] == 1))   # True Positive (correctly gained)
    )

    accuracy = data['correct_prediction'].mean()

    y_true = (data['daily_return'] > 0).astype(int) 
    y_pred = data['position'] 

    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()

    report = classification_report(y_true, y_pred, target_names=['Avoid Loss (0)', 'Capture Gain (1)'])

    print("\nSTRATEGY PERFORMANCE:")
    print(f"Accuracy: {accuracy:.2%}")
    print("\nConfusion Matrix:")
    print(cm)
    print(f"\nTrue Negatives (Avoid Loss Correctly): {tn}")
    print(f"False Positives (Invested but Lost): {fp}")
    print(f"False Negatives (Missed Gains): {fn}")
    print(f"True Positives (Captured Gains): {tp}")
    print("\nClassification Report:")
    print(report)

    data['strategy_return'] = data['position'].shift(1) * data['daily_return']
    data['cumulative_strategy'] = data['strategy_return'].cumsum()
    data['perfect_return'] = data['daily_return'].where(data['daily_return'] > 0, 0)
    data['cumulative_perfect'] = data['perfect_return'].cumsum()
    
    total_strategy = data['cumulative_strategy'].iloc[-1]
    total_perfect = data['cumulative_perfect'].iloc[-1]
    efficiency = total_strategy / total_perfect if total_perfect > 0 else 0
    
    print("\nENHANCED SMA STRATEGY")
    print(f"Parameters: Threshold={threshold_pct}%, Min Slope={min_slope}%")
    print(f"Strategy Return: {total_strategy:.2f}")
    print(f"Perfect Return: {total_perfect:.2f}")
    print(f"Efficiency: {efficiency:.2%}")
    print(f"Days in Market: {data['position'].sum()}/{len(data)}")
    
    return data




In [127]:
sma_results = enhanced_sma_strategy(
    '../data/individual/moving_avgs_data.csv',
    '../data/joined/full_dataset.csv',
    threshold_pct=0.09,  
    min_slope=0.35
)


STRATEGY PERFORMANCE:
Accuracy: 45.59%

Confusion Matrix:
[[ 537  197]
 [1586  957]]

True Negatives (Avoid Loss Correctly): 537
False Positives (Invested but Lost): 197
False Negatives (Missed Gains): 1586
True Positives (Captured Gains): 957

Classification Report:
                  precision    recall  f1-score   support

  Avoid Loss (0)       0.25      0.73      0.38       734
Capture Gain (1)       0.83      0.38      0.52      2543

        accuracy                           0.46      3277
       macro avg       0.54      0.55      0.45      3277
    weighted avg       0.70      0.46      0.49      3277


ENHANCED SMA STRATEGY
Parameters: Threshold=0.09%, Min Slope=0.35%
Strategy Return: 369.58
Perfect Return: 974.40
Efficiency: 37.93%
Days in Market: 1154/3277


In [128]:
# Run the strategy
result1 = golden_cross_strategy1(
    '../data/individual/moving_avgs_data.csv',
    '../data/joined/full_dataset.csv',
    session_length=7
)




STRATEGY PERFORMANCE:
Accuracy: 77.91%

Confusion Matrix:
[[  10  724]
 [   0 2543]]

True Negatives (Avoid Loss Correctly): 10
False Positives (Invested but Lost): 724
False Negatives (Missed Gains): 0
True Positives (Captured Gains): 2543

Classification Report:
                  precision    recall  f1-score   support

  Avoid Loss (0)       1.00      0.01      0.03       734
Capture Gain (1)       0.78      1.00      0.88      2543

        accuracy                           0.78      3277
       macro avg       0.89      0.51      0.45      3277
    weighted avg       0.83      0.78      0.69      3277


STRATEGY PERFORMANCE:
Strategy Total Return: 911.31
Perfect Strategy Total Return: 974.40
Efficiency (Strategy/Perfect): 93.53%
Days in Market: 3267/3277


In [129]:
# Run the strategy
result2 = golden_cross_strategy2(
    '../data/individual/moving_avgs_data.csv',
    '../data/joined/full_dataset.csv',
    session_length=8
)


STRATEGY PERFORMANCE:
Accuracy: 53.77%

Confusion Matrix:
[[ 452  282]
 [1233 1310]]

True Negatives (Avoid Loss Correctly): 452
False Positives (Invested but Lost): 282
False Negatives (Missed Gains): 1233
True Positives (Captured Gains): 1310

Classification Report:
                  precision    recall  f1-score   support

  Avoid Loss (0)       0.27      0.62      0.37       734
Capture Gain (1)       0.82      0.52      0.63      2543

        accuracy                           0.54      3277
       macro avg       0.55      0.57      0.50      3277
    weighted avg       0.70      0.54      0.58      3277


REVISED STRATEGY PERFORMANCE:
Total Trading Days: 3277
Days in Position: 1592 (48.6%)
Strategy Return: 538.50
Perfect Strategy Return: 974.40
Efficiency: 55.26%
Max Consecutive Days in Position: 312


In [130]:
# Save or display results
print(result2[['datetime', 'signal', 'position', 'cumulative_strategy', 'cumulative_perfect' ]].tail(20))
result2.to_csv('../data/strategy_results/golden_cross_sma_results.csv', index=False)

                datetime  signal  position  cumulative_strategy  \
3257 2025-03-28 14:30:00       0         0           537.723262   
3258 2025-03-28 15:30:00       0         0           537.723262   
3259 2025-03-28 16:30:00       0         0           537.723262   
3260 2025-03-28 17:30:00       0         0           537.723262   
3261 2025-03-28 18:30:00       0         0           537.723262   
3262 2025-03-28 19:30:00       0         0           537.723262   
3263 2025-03-31 13:30:00       0         0           537.723262   
3264 2025-03-31 14:30:00       0         0           537.723262   
3265 2025-03-31 15:30:00       0         0           537.723262   
3266 2025-03-31 16:30:00       0         0           537.723262   
3267 2025-03-31 17:30:00       0         0           537.723262   
3268 2025-03-31 18:30:00       0         0           537.723262   
3269 2025-03-31 19:30:00       0         0           537.723262   
3270 2025-04-01 13:30:00       0         0           537.72326

In [131]:
def ema_golden_cross_strategy1(moving_avgs_file, full_dataset_file, session_length=7):
    moving_avgs = pd.read_csv(moving_avgs_file, parse_dates=['datetime'])
    full_data = pd.read_csv(full_dataset_file, parse_dates=['datetime'])
    data = pd.merge(moving_avgs, full_data[['datetime', 'Cumulative_Return']], 
                   on='datetime', how='left')
    

    data['daily_return'] = data['Cumulative_Return']
    data['ema5_slope'] = data['EMA_5'].diff()
    data['ema20_slope'] = data['EMA_20'].diff()
    

    data['signal'] = 0
    data['position'] = 0
    data['consecutive_sells'] = 0
    

    for i in range(2, len(data)):
        ema5 = data.at[i, 'EMA_5']
        ema20 = data.at[i, 'EMA_20']
        ema50 = data.at[i, 'EMA_50']
        slope5 = data.at[i, 'ema5_slope']
        slope20 = data.at[i, 'ema20_slope']
        

        buy_conditions = [

            (ema5 > ema20 > ema50),
            (ema5 > ema20) & (ema20 > ema50) & (slope5 > 0) & (slope20 > 0),
            (ema5 > ema20) & (data.at[i-1, 'EMA_5'] < data.at[i-1, 'EMA_20']) & (slope5 > abs(slope20))
        ]
        
        sell_conditions = [
            (ema5 < ema20),
            (slope5 < 0),
            (data.at[i, 'daily_return'] < 0)
        ]
        
        if any(buy_conditions) and not any(sell_conditions):
            data.at[i, 'signal'] = 1
            data.at[i, 'consecutive_sells'] = 0
        else:
            data.at[i, 'signal'] = 0
            data.at[i, 'consecutive_sells'] = data.at[i-1, 'consecutive_sells'] + 1
    
    in_session = False
    for i in range(1, len(data)):
        if not in_session:
            if data.at[i, 'signal'] == 1:
                in_session = True
                data.at[i, 'position'] = 1
        else:
            if (data.at[i, 'consecutive_sells'] >= 3) or \
               ((i - data[data['position'].diff() == 1].index[-1]) >= session_length):
                in_session = False
                data.at[i, 'position'] = 0
            else:
                data.at[i, 'position'] = 1
    

    data['correct_prediction'] = (
        ((data['daily_return'] < 0) & (data['position'] == 0)) |  # True Negative (correctly avoided loss)
        ((data['daily_return'] > 0) & (data['position'] == 1))   # True Positive (correctly gained)
    )

    accuracy = data['correct_prediction'].mean()

    y_true = (data['daily_return'] > 0).astype(int) 
    y_pred = data['position'] 

    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()

    report = classification_report(y_true, y_pred, target_names=['Avoid Loss (0)', 'Capture Gain (1)'])

    print("\nSTRATEGY PERFORMANCE:")
    print(f"Accuracy: {accuracy:.2%}")
    print("\nConfusion Matrix:")
    print(cm)
    print(f"\nTrue Negatives (Avoid Loss Correctly): {tn}")
    print(f"False Positives (Invested but Lost): {fp}")
    print(f"False Negatives (Missed Gains): {fn}")
    print(f"True Positives (Captured Gains): {tp}")
    print("\nClassification Report:")
    print(report)

    data['strategy_return'] = data['position'].shift(1) * data['daily_return']
    data['cumulative_strategy'] = data['strategy_return'].cumsum()
    data['perfect_return'] = data['daily_return'].where(data['daily_return'] > 0, 0)
    data['cumulative_perfect'] = data['perfect_return'].cumsum()
    
    total_strategy = data['cumulative_strategy'].iloc[-1]
    total_perfect = data['cumulative_perfect'].iloc[-1]
    efficiency = total_strategy / total_perfect if total_perfect > 0 else 0
    
    print("\nENHANCED EMA STRATEGY")
    print(f"Strategy Total Return: {total_strategy:.2f}")
    print(f"Perfect Strategy Total Return: {total_perfect:.2f}")
    print(f"Efficiency (Strategy/Perfect): {efficiency:.2%}")
    print(f"Days in Market: {data['position'].sum()}/{len(data)}")
    
    return data



In [132]:
def ema_golden_cross_strategy2(moving_avgs_file, full_dataset_file, session_length=7):
    moving_avgs = pd.read_csv(moving_avgs_file, parse_dates=['datetime'])
    full_data = pd.read_csv(full_dataset_file, parse_dates=['datetime'])
    merged_data = pd.merge(moving_avgs, full_data[['datetime', 'Cumulative_Return']], 
                          on='datetime', how='left')
    

    merged_data['daily_return'] = merged_data['Cumulative_Return']
    

    merged_data['signal'] = 0
    merged_data['position'] = 0
    merged_data['consecutive_sells'] = 0
    

    in_session = False
    
    for i in range(1, len(merged_data)):
        ema5 = merged_data.at[i, 'EMA_5']
        ema20 = merged_data.at[i, 'EMA_20']
        ema50 = merged_data.at[i, 'EMA_50']
        
        # Determine signal
        buy_condition = (
            (ema5 > ema20 > ema50) or 
            (ema20 > ema5 > ema50) or 
            (ema5 > ema50 > ema20)
        )
        
        if buy_condition:
            merged_data.at[i, 'signal'] = 1
            merged_data.at[i, 'consecutive_sells'] = 0
        else:
            merged_data.at[i, 'signal'] = 0
            merged_data.at[i, 'consecutive_sells'] = merged_data.at[i-1, 'consecutive_sells'] + 1
        

        if not in_session:
            if buy_condition:
                in_session = True
                merged_data.at[i, 'position'] = 1
        else:
            if (merged_data.at[i, 'consecutive_sells'] >= 3) or \
               ((i - merged_data[merged_data['position'].diff() == 1].index[-1]) >= session_length):
                in_session = False
                merged_data.at[i, 'position'] = 0
            else:
                merged_data.at[i, 'position'] = 1
    

    merged_data['correct_prediction'] = (
        ((merged_data['daily_return'] < 0) & (merged_data['position'] == 0)) |  # True Negative (correctly avoided loss)
        ((merged_data['daily_return'] > 0) & (merged_data['position'] == 1))   # True Positive (correctly gained)
    )

    accuracy = merged_data['correct_prediction'].mean()

    y_true = (merged_data['daily_return'] > 0).astype(int) 
    y_pred = merged_data['position'] 

    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()

    report = classification_report(y_true, y_pred, target_names=['Avoid Loss (0)', 'Capture Gain (1)'])

    print("\nSTRATEGY PERFORMANCE:")
    print(f"Accuracy: {accuracy:.2%}")
    print("\nConfusion Matrix:")
    print(cm)
    print(f"\nTrue Negatives (Avoid Loss Correctly): {tn}")
    print(f"False Positives (Invested but Lost): {fp}")
    print(f"False Negatives (Missed Gains): {fn}")
    print(f"True Positives (Captured Gains): {tp}")
    print("\nClassification Report:")
    print(report)

    merged_data['strategy_return'] = merged_data['position'].shift(1) * merged_data['daily_return']
    merged_data['cumulative_strategy'] = merged_data['strategy_return'].cumsum()
    

    merged_data['perfect_return'] = merged_data['daily_return'].where(merged_data['daily_return'] > 0, 0)
    merged_data['cumulative_perfect'] = merged_data['perfect_return'].cumsum()
    

    total_strategy = merged_data['cumulative_strategy'].iloc[-1]
    total_perfect = merged_data['cumulative_perfect'].iloc[-1]
    efficiency = total_strategy / total_perfect if total_perfect > 0 else 0
    
    print("\nSTRATEGY PERFORMANCE:")
    print(f"Strategy Total Return: {total_strategy:.2f}")
    print(f"Perfect Strategy Total Return: {total_perfect:.2f}")
    print(f"Efficiency (Strategy/Perfect): {efficiency:.2%}")
    print(f"Days in Market: {merged_data['position'].sum()}/{len(merged_data)}")
    
    return merged_data



In [133]:
# Run the strategy
ema_results1 = ema_golden_cross_strategy1(
    '../data/individual/moving_avgs_data.csv',
    '../data/joined/full_dataset.csv',
    session_length=8
)



STRATEGY PERFORMANCE:
Accuracy: 52.76%

Confusion Matrix:
[[ 727    7]
 [1541 1002]]

True Negatives (Avoid Loss Correctly): 727
False Positives (Invested but Lost): 7
False Negatives (Missed Gains): 1541
True Positives (Captured Gains): 1002

Classification Report:
                  precision    recall  f1-score   support

  Avoid Loss (0)       0.32      0.99      0.48       734
Capture Gain (1)       0.99      0.39      0.56      2543

        accuracy                           0.53      3277
       macro avg       0.66      0.69      0.52      3277
    weighted avg       0.84      0.53      0.55      3277


ENHANCED EMA STRATEGY
Strategy Total Return: 388.94
Perfect Strategy Total Return: 974.40
Efficiency (Strategy/Perfect): 39.92%
Days in Market: 1009/3277


In [134]:
# Run the strategy
ema_results1 = ema_golden_cross_strategy2(
    '../data/individual/moving_avgs_data.csv',
    '../data/joined/full_dataset.csv',
    session_length=8
)



STRATEGY PERFORMANCE:
Accuracy: 50.66%

Confusion Matrix:
[[ 380  354]
 [1263 1280]]

True Negatives (Avoid Loss Correctly): 380
False Positives (Invested but Lost): 354
False Negatives (Missed Gains): 1263
True Positives (Captured Gains): 1280

Classification Report:
                  precision    recall  f1-score   support

  Avoid Loss (0)       0.23      0.52      0.32       734
Capture Gain (1)       0.78      0.50      0.61      2543

        accuracy                           0.51      3277
       macro avg       0.51      0.51      0.47      3277
    weighted avg       0.66      0.51      0.55      3277


STRATEGY PERFORMANCE:
Strategy Total Return: 486.63
Perfect Strategy Total Return: 974.40
Efficiency (Strategy/Perfect): 49.94%
Days in Market: 1634/3277


In [135]:
def threshold_ema_strategy(moving_avgs_file, full_dataset_file, session_length=7, threshold_pct=0.5, min_slope=0.1):

    data = pd.merge(
        pd.read_csv(moving_avgs_file, parse_dates=['datetime']),
        pd.read_csv(full_dataset_file, parse_dates=['datetime'])[['datetime', 'Cumulative_Return']],
        on='datetime'
    )
    
    data['daily_return'] = data['Cumulative_Return']
    data['ema5_slope'] = data['EMA_5'].pct_change() * 100  
    data['ema20_slope'] = data['EMA_20'].pct_change() * 100
    
    data['signal'] = 0
    data['position'] = 0
    data['consecutive_sells'] = 0
    
    for i in range(2, len(data)):
        ema5 = data.at[i, 'EMA_5']
        ema20 = data.at[i, 'EMA_20']
        ema50 = data.at[i, 'EMA_50']
        slope5 = data.at[i, 'ema5_slope']
        slope20 = data.at[i, 'ema20_slope']
        prev_ema5 = data.at[i-1, 'EMA_5']
        prev_ema20 = data.at[i-1, 'EMA_20']
        
        ema5_above_20 = (ema5 > ema20 * (1 + threshold_pct/100))
        ema20_above_50 = (ema20 > ema50 * (1 + threshold_pct/100))
        strong_slope = (slope5 > min_slope) and (slope20 > min_slope/2)
        crossing_up = (ema5 > ema20) and (prev_ema5 <= prev_ema20)
        
        buy_signal = sum([
            ema5_above_20,
            ema20_above_50, 
            strong_slope,
            crossing_up
        ]) >= 2
        
        ema5_below_20 = (ema5 < ema20 * (1 - threshold_pct/100))
        negative_slope = (slope5 < -min_slope)
        
        if buy_signal and not (ema5_below_20 or negative_slope):
            data.at[i, 'signal'] = 1
            data.at[i, 'consecutive_sells'] = 0
        else:
            data.at[i, 'signal'] = 0
            data.at[i, 'consecutive_sells'] = data.at[i-1, 'consecutive_sells'] + 1
    
    in_session = False
    for i in range(1, len(data)):
        if not in_session:
            if data.at[i, 'signal'] == 1:
                in_session = True
                data.at[i, 'position'] = 1
        else:
            if (data.at[i, 'consecutive_sells'] >= 3) or \
               ((i - data[data['position'].diff() == 1].index[-1]) >= session_length):
                in_session = False
                data.at[i, 'position'] = 0
            else:
                data.at[i, 'position'] = 1
    
    data['correct_prediction'] = (
        ((data['daily_return'] < 0) & (data['position'] == 0)) |  # True Negative (correctly avoided loss)
        ((data['daily_return'] > 0) & (data['position'] == 1))   # True Positive (correctly gained)
    )

    accuracy = data['correct_prediction'].mean()

    y_true = (data['daily_return'] > 0).astype(int) 
    y_pred = data['position'] 

    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()

    report = classification_report(y_true, y_pred, target_names=['Avoid Loss (0)', 'Capture Gain (1)'])

    print("\nSTRATEGY PERFORMANCE:")
    print(f"Accuracy: {accuracy:.2%}")
    print("\nConfusion Matrix:")
    print(cm)
    print(f"\nTrue Negatives (Avoid Loss Correctly): {tn}")
    print(f"False Positives (Invested but Lost): {fp}")
    print(f"False Negatives (Missed Gains): {fn}")
    print(f"True Positives (Captured Gains): {tp}")
    print("\nClassification Report:")
    print(report)

    data['strategy_return'] = data['position'].shift(1) * data['daily_return']
    data['cumulative_strategy'] = data['strategy_return'].cumsum()
    data['perfect_return'] = data['daily_return'].where(data['daily_return'] > 0, 0)
    data['cumulative_perfect'] = data['perfect_return'].cumsum()
    
    total_strategy = data['cumulative_strategy'].iloc[-1]
    total_perfect = data['cumulative_perfect'].iloc[-1]
    efficiency = total_strategy / total_perfect if total_perfect > 0 else 0
    
    print("\nTHRESHOLD-BASED EMA STRATEGY")
    print(f"Parameters: Threshold={threshold_pct}%, Min Slope={min_slope}%")
    print(f"Strategy Return: {total_strategy:.2f}")
    print(f"Perfect Return: {total_perfect:.2f}")
    print(f"Efficiency: {efficiency:.2%}")
    print(f"Days in Market: {data['position'].sum()}/{len(data)}")
    
    return data


In [136]:
ema_results2 = threshold_ema_strategy(
    '../data/individual/moving_avgs_data.csv',
    '../data/joined/full_dataset.csv',
    threshold_pct=0.3,
    min_slope=0.3
)


STRATEGY PERFORMANCE:
Accuracy: 47.60%

Confusion Matrix:
[[ 505  229]
 [1488 1055]]

True Negatives (Avoid Loss Correctly): 505
False Positives (Invested but Lost): 229
False Negatives (Missed Gains): 1488
True Positives (Captured Gains): 1055

Classification Report:
                  precision    recall  f1-score   support

  Avoid Loss (0)       0.25      0.69      0.37       734
Capture Gain (1)       0.82      0.41      0.55      2543

        accuracy                           0.48      3277
       macro avg       0.54      0.55      0.46      3277
    weighted avg       0.69      0.48      0.51      3277


THRESHOLD-BASED EMA STRATEGY
Parameters: Threshold=0.3%, Min Slope=0.3%
Strategy Return: 405.71
Perfect Return: 974.40
Efficiency: 41.64%
Days in Market: 1284/3277


In [137]:

# Save results
ema_results2.to_csv('../data/strategy_results/ema_golden_cross_results.csv', index=False)